In [145]:
# Importing libraries 
import pandas as pd  
import numpy as np  

In [146]:
phishing = pd.read_csv("C:\\Users\\HP\\Downloads\\archive (1)\\phishing.csv")

In [147]:
from sklearn.model_selection import train_test_split, cross_val_score

In [148]:
#creating input class and target 
X = phishing.iloc[:, 1:-1]  # Select all columns except the first and last

Y = phishing['class']
Y = pd.DataFrame(Y)

X.head()


,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,Favicon,...,DisableRightClick,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport
0,1,1,1,1,1,-1,0,1,-1,1,...,1,1,1,-1,-1,0,-1,1,1,1
1,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,1,-1,1,-1,1,0,-1
2,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,1,-1,-1,1,-1,1,-1,1
3,1,0,-1,1,1,-1,1,1,-1,1,...,1,-1,1,-1,-1,0,-1,1,1,1
4,-1,0,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,1,1,-1,1,-1,-1


In [149]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state=2)

In [150]:
from sklearn.tree import DecisionTreeClassifier

In [164]:
dtree = DecisionTreeClassifier()
model_3 = dtree.fit(train_X, train_Y)


In [165]:
dtree_predict = model_3.predict(test_X)

In [166]:

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [167]:
round(accuracy_score(dtree_predict, test_Y),4)

0.9629

In [168]:
print(model_3.n_features_in_)  # Number of input features


30


In [169]:
print(model_3.feature_names_in_)  # Feature names used in training


['UsingIP' 'LongURL' 'ShortURL' 'Symbol@' 'Redirecting//' 'PrefixSuffix-'
 'SubDomains' 'HTTPS' 'DomainRegLen' 'Favicon' 'NonStdPort'
 'HTTPSDomainURL' 'RequestURL' 'AnchorURL' 'LinksInScriptTags'
 'ServerFormHandler' 'InfoEmail' 'AbnormalURL' 'WebsiteForwarding'
 'StatusBarCust' 'DisableRightClick' 'UsingPopupWindow'
 'IframeRedirection' 'AgeofDomain' 'DNSRecording' 'WebsiteTraffic'
 'PageRank' 'GoogleIndex' 'LinksPointingToPage' 'StatsReport']


In [188]:

import re
import socket
import requests
import whois
import datetime
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import urlparse

# API Keys (Replace with real keys)
VIRUSTOTAL_API_KEY = "c4f5a97033cd6aa4116c9ecf0b4f0c97c5da8e0bd5240adf4c36e548c3b226bb"
SIMILARWEB_API_KEY = "your_similarweb_api_key"
OPENPAGERANK_API_KEY = "s4o4sc8cokcg0ks884csgskgsgo4s4swcg00kkgk "

# Function to extract features from a URL
def extract_url_features(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
    except:
        response = None
        soup = None

    domain = urlparse(url).netloc
    features = []

    # 1️⃣-7️⃣ URL & Domain-based Features
    features.append(1 if re.match(r'\d+\.\d+\.\d+\.\d+', domain) else -1)  # Using IP
    features.append(1 if len(url) > 75 else -1)  # Long URL
    features.append(1 if len(url) < 15 else -1)  # Short URL
    features.append(1 if '@' in url else -1)  # Symbol @
    features.append(1 if url.count('//') > 1 else -1)  # Redirecting //
    features.append(1 if '-' in domain else -1)  # Prefix Suffix -
    features.append(-1 if domain.count('.') == 1 else (0 if domain.count('.') == 2 else 1))  # Sub Domains

    # 8️⃣-1️⃣2️⃣ Security & Domain Registration
    features.append(1 if url.startswith("https") else -1)  # HTTPS
    
    # Domain registration length
    try:
        domain_info = whois.whois(domain)
        expiration_date = domain_info.expiration_date
        expiration_date = expiration_date[0] if isinstance(expiration_date, list) else expiration_date
        domain_reg_len = (expiration_date - datetime.datetime.now()).days if expiration_date else 0
    except:
        domain_reg_len = 0
    features.append(1 if domain_reg_len >= 365 else -1)  # Domain Registration Length
    
    features.append(1 if soup and soup.find("link", rel="shortcut icon") else -1)  # Favicon
    features.append(1 if ':' in domain else -1)  # Non-Standard Port
    features.append(1 if "https" in domain else -1)  # HTTPS in Domain

    # 1️⃣3️⃣-1️⃣8️⃣ External Links & Scripts
    try:
        img_count = len(soup.find_all("img"))
        external_img_count = sum(1 for img in soup.find_all("img") if urlparse(img.get("src", "")).netloc != domain)
        features.append(-1 if img_count and (external_img_count / img_count) > 0.5 else 1)  # Request URL
    except:
        features.append(-1)

    try:
        a_tags = soup.find_all("a")
        external_anchors = sum(1 for a in a_tags if urlparse(a.get("href", "")).netloc != domain)
        external_ratio = external_anchors / len(a_tags) if a_tags else 0
        features.append(-1 if external_ratio > 0.67 else (0 if external_ratio > 0.33 else 1))  # Anchor URL
    except:
        features.append(-1)

    try:
        scripts = soup.find_all("script")
        external_scripts = sum(1 for s in scripts if s.has_attr("src") and urlparse(s["src"]).netloc != domain)
        features.append(-1 if scripts and (external_scripts / len(scripts)) > 0.5 else 1)  # Links In Script Tags
    except:
        features.append(-1)

    try:
        forms = soup.find_all("form")
        external_forms = any(form.has_attr("action") and urlparse(form["action"]).netloc != domain for form in forms)
        features.append(-1 if external_forms else 1)  # Server Form Handler
    except:
        features.append(-1)

    features.append(1 if "mailto:" in url else -1)  # Info Email
    
    try:
        features.append(1 if whois.whois(domain).domain_name else -1)  # Abnormal URL
    except:
        features.append(-1)

    features.append(1 if response and len(response.history) > 2 else -1)  # Website Forwarding

    # 1️⃣9️⃣-2️⃣6️⃣ JavaScript & Domain Age
    try:
        features.append(-1 if any("window.status" in script.text for script in scripts) else 1)  # Status Bar Cust
        features.append(-1 if any("event.button == 2" in script.text for script in scripts) else 1)  # Disable Right Click
        features.append(-1 if any("window.open" in script.text for script in scripts) else 1)  # Using Popup Window
    except:
        features.append(-1)

    features.append(-1 if soup and soup.find_all("iframe") else 1)  # Iframe Redirection

    try:
        age_of_domain = (datetime.datetime.now() - domain_info.creation_date).days if domain_info.creation_date else 0
    except:
        age_of_domain = 0
    features.append(1 if age_of_domain >= 365 else -1)  # Age of Domain

    try:
        features.append(1 if socket.gethostbyname(domain) else -1)  # DNS Recording
    except:
        features.append(-1)

    # External API-based Features (Ensure default values if API fails)
    try:
        website_traffic = -1  # Placeholder (Replace with actual API call if needed)
        features.append(website_traffic)
    except:
        features.append(-1)

    try:
        page_rank = -1  # Placeholder (Replace with actual API call if needed)
        features.append(page_rank)
    except:
        features.append(-1)

    try:
        google_indexed = -1  # Placeholder (Replace with actual API call if needed)
        features.append(google_indexed)
    except:
        features.append(-1)

    try:
        backlinks = -1  # Placeholder (Replace with actual API call if needed)
        features.append(backlinks)
    except:
        features.append(-1)

    try:
        malware_check = -1  # Placeholder (Replace with actual API call if needed)
        features.append(malware_check)
    except:
        features.append(-1)

    # ✅ Ensure fixed feature vector size
    while len(features) < 30:
        features.append(0)  # Add placeholder values for missing features

    return np.array(features).reshape(1, -1)

# Example Usage
url = "https://www.thapar.edu/academics/centers/thapar-lms65"
features = extract_url_features(url)
print(features)



[[-1 -1 -1 -1 -1 -1  0  1  1  1 -1 -1 -1  0 -1 -1 -1  1 -1  1  1  1 -1  1
   1 -1 -1 -1 -1 -1]]


In [187]:
import joblib

# Save the model as a file
joblib.dump(model, "model_3.joblib")

# Load the saved model
model = joblib.load("model_3.joblib")


# Extract features
X_input = extract_url_features(url)

prediction = model.predict(X_input)[0]
print(f"Prediction: {'Fake' if prediction == 1 else 'Legit'}")




Prediction: Legit


C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [144]:
import numpy as np
unique, counts = np.unique(train_Y, return_counts=True)
print(dict(zip(unique, counts)))


{-1: 3931, 1: 4912}


In [189]:
# example two
url = ""
features = extract_url_features(url)
print(features)
import joblib

# Save the model as a file
joblib.dump(model, "model_3.joblib")

# Load the saved model
model = joblib.load("model_3.joblib")


# Extract features
X_input = extract_url_features(url)

prediction = model.predict(X_input)[0]
print(f"Prediction: {'Fake' if prediction == 1 else 'Legit'}")


[[-1 -1 -1 -1 -1 -1  0  1  1  1 -1 -1 -1  0 -1 -1 -1  1 -1  1  1  1 -1  1
   1 -1 -1 -1 -1 -1]]
Prediction: Fake


C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
